<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/two_concepts/training_two_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
get_ipython().system('pip install -q git+https://github.com/keras-team/keras-cv.git')
get_ipython().system('pip install -q tensorflow==2.11.0')
get_ipython().system('pip install pyyaml h5py')

In [ ]:
!pip install keras_cv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.9/634.9 KB 24.4 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/tensorflow-project/FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 1938, done.
remote: Counting objects: 100% (901/901), done.
remote: Compressing objects: 100% (293/293), done.
remote: Total 1938 (delta 667), reused 821 (delta 603), pack-reused 1037
Receiving objects: 100% (1938/1938), 163.51 MiB | 24.78 MiB/s, done.
Resolving deltas: 100% (1386/1386), done.


In [ ]:
import sys
import os
import numpy as np
from google.colab import drive
import keras
import keras_cv
import tensorflow as tf
import urllib.request

### agree to mounting your Google Drive
drive.mount("/content/drive")

py_file_location = "/content/FineTuning/two_concepts"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

import textual_inversion_two_concepts as txt

You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.
Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 96.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
### create an instance of the Stable Diffusion model
stable_diffusion = txt.StableDiffusion()

### our new concept which is later inserted in the different prompts (for training and image generation)
placeholder_token_broccoli = "<my-broccoli-token>"
placeholder_token_emoji = "<my-emoji-token>"
placeholder_token_combined = "<my-broccoli-emoji-token>"

### Add our placeholder_tokens to our stable_diffusion Model
stable_diffusion.tokenizer.add_tokens(placeholder_token_broccoli)
stable_diffusion.tokenizer.add_tokens(placeholder_token_emoji)
stable_diffusion.tokenizer.add_tokens(placeholder_token_combined)

train_ds = txt.create_dataset(stable_diffusion, placeholder_token_broccoli, placeholder_token_emoji)

txt.textual_preprocessing(stable_diffusion, placeholder_token_broccoli, placeholder_token_emoji, placeholder_token_combined)

### beta is the diffusion rate
noise_scheduler = txt.NoiseScheduler(
    ### beta_start determines the amount of noise added at the start of the denoising process
    beta_start=0.00085,
    ### beta_end at the end of the denoising process
    beta_end=0.012,
    ### the beta_schedule determines that the diffusion rate increases linearly
    beta_schedule="scaled_linear",
    train_timesteps=1000,
)

### Initialize the model we use to fine tune our concept
trainer = txt.StableDiffusionFineTuner(stable_diffusion, noise_scheduler, name="trainer")
#t = txt.StableDiffusionFineTuner(stable_diffusion, noise_scheduler, name="t")


#EPOCHS = 50
### learning rate decays depending on the number of epochs to avoid convergence issues in few epochs 
### in the originial tutorial a scheduler is used but we experienced to have better results without a scheduler
"""learning_rate = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-4, decay_steps=train_ds.cardinality() * EPOCHS
)"""
### inizialize the optimizer
optimizer = tf.keras.optimizers.Adam(
    weight_decay=0.004, learning_rate=1e-4, epsilon=1e-8, global_clipnorm=10
)

trainer.compile(
    optimizer=optimizer,
    # We are performing reduction manually in our train step, so none is required here.
    loss=keras.losses.MeanSquaredError(reduction="none"),
)  

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
13911/13911 [==============================] - 0s 0us/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


136824240/136824240 [==============================] - 1s 0us/step


In [ ]:
### EXECUTE THE FOLLOWING TWO BLOCKS ONLY IF YOU WANT TO CONTINUE TRAINING WITH SAVED WEIGTHS
### for downloading weights from Dropbox:
### if you want to use any other service than Dropbox, change the code underneath

### Replace "" and "" with the ID and name of your file
### The file id is the string of characters between "s/" and the next slash "/"
### example: https://www.dropbox.com/s/yttja9ihaoq2xhd/35epochs_weights_with_two_concepts.npy?dl=0
file_id = 'yttja9ihaoq2xhd'

### name óf file
file_name = '35epochs_weights_with_two_concepts.npy'
url = f'https://www.dropbox.com/s/{file_id}/{file_name}?dl=1'
filename = f'{file_name}.npy'

In [ ]:


### Download the file from the URL and save it locally
urllib.request.urlretrieve(url, filename=filename)

### Load the saved numpy file using numpy.load()
weights = np.load(filename, allow_pickle=True)

### Set the weights of the text encoder
stable_diffusion.text_encoder.set_weights(weights)

In [ ]:
### training
trainer.fit(
    train_ds,
    epochs=1,
)

177/220 [=======================>......] - ETA: 1:07 - loss: 0.0275

In [ ]:
### choose where to save your newly generated weights
path = '/content/drive/MyDrive/weight_with_two_concepts.npy'

In [ ]:
###get the weights of the text encoder and save the to Google Drive
text_encoder_weights = np.array(stable_diffusion.text_encoder.get_weights(), dtype=object)

### Save the weights array to a file on your Google Drive
np.save(path, text_encoder_weights)